Подключение модулей

In [1]:
# Набор данных
from tensorflow.keras.datasets import fashion_mnist
# Модель нейронных сетей
from tensorflow.keras.models import Sequential
# Слои
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras import utils
# Предварительно обученная модель сети
from tensorflow.keras.applications import Xception
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

Загрузка тренировочных и тестовых данных

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


Преобразование изображения в 3 канала

In [3]:
x_train = np.dstack([x_train]*3)
x_test = np.dstack([x_test]*3)

Изменение формы изображений в соответствии с тензорным форматом

In [4]:
x_train = x_train.reshape(-1,28,28,3)
x_test = x_test.reshape(-1,28,28,3)

Изменение размера изображений 71x71 в соответствии с требованиями Xception

In [5]:
x_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((71, 71))) for im in x_train])
x_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((71, 71))) for im in x_test])

Нормализация данных

In [6]:
assert x_train.shape == (60000, 71, 71, 3)
assert x_test.shape == (10000, 71, 71, 3)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

Преобразование меток в категории

In [7]:
y_train = utils.to_categorical(y_train,10)
y_test = utils.to_categorical(y_test,10)

Создание нейронной сети

In [8]:
# include_top = false - загрузка только сверточной части сети
# weights='imagenet' - предварительно обученные веса
# input_shape - размер тензора
mobileNet = Xception(weights='imagenet', include_top=False, input_shape=(71,71,3))
# Без обучения сверточной части сети
mobileNet.trainable = False

model = Sequential()
# Добавление сверточного слоя
model.add(mobileNet)
# Преобразование двумерного массива в одномерный
model.add(Flatten())
# Добавление полносвязного слоя. 
# На вход: 800 нейронов
model.add(Dense(800, activation='relu'))
# Слой регуляризации
model.add(Dropout(0.5))
# На выход: 10 нейронов
# softmax обеспечивает сумму вероятности, 
# равную единицы на выходе всех нейронов
model.add(Dense(10, activation='softmax'))

83697664/83683744 [==============================] - 1s 0us/step


Компиляция модели

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='SGD', 
              metrics=['accuracy'])

Вывод параметров модели

In [10]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 800)               14746400  
_________________________________________________________________
dropout (Dropout)            (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                8010      
Total params: 35,615,890
Trainable params: 14,754,410
Non-trainable params: 20,861,480
_________________________________________________________________
None


Обучение сети

In [11]:
# На каждой эпохе подается 200 изображений
model.fit(x_train, y_train, 
          batch_size=200, 
          epochs=15, 
          validation_split=0.1,
          shuffle=True)

Epoch 1/15
270/270 [==============================] - 1148s 4s/step - loss: 0.8588 - accuracy: 0.7126 - val_loss: 0.5681 - val_accuracy: 0.8082
Epoch 2/15
270/270 [==============================] - 1144s 4s/step - loss: 0.5940 - accuracy: 0.7914 - val_loss: 0.5219 - val_accuracy: 0.8137
Epoch 3/15
270/270 [==============================] - 1144s 4s/step - loss: 0.5386 - accuracy: 0.8099 - val_loss: 0.5001 - val_accuracy: 0.8233
Epoch 4/15
270/270 [==============================] - 1144s 4s/step - loss: 0.5030 - accuracy: 0.8214 - val_loss: 0.4620 - val_accuracy: 0.8340
Epoch 5/15
270/270 [==============================] - 1144s 4s/step - loss: 0.4810 - accuracy: 0.8282 - val_loss: 0.4552 - val_accuracy: 0.8357
Epoch 6/15
270/270 [==============================] - 1144s 4s/step - loss: 0.4633 - accuracy: 0.8352 - val_loss: 0.4440 - val_accuracy: 0.8405
Epoch 7/15
270/270 [==============================] - 1144s 4s/step - loss: 0.4441 - accuracy: 0.8409 - val_loss: 0.4377 - val_accuracy:

Проверка качества обучения сети на тестовой выборке

In [12]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('Доля верных ответов на тестовых данных, в процентах:', round(scores[1]*100, 4))

Доля верных ответов на тестовых данных, в процентах: 85.04
